In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#lots of question marks in the data so used the argument na_values =['?']
#object dtypes except the first 2 column gets converted to float type
data = pd.read_csv("../input/electrity-prices/electricity_prices.csv", na_values=['?'])

## description of the rows

* DateTime: String, defines date and time of sample
* Holiday: String, gives name of holiday if day is a bank holiday
* HolidayFlag: integer, 1 if day is a bank holiday, zero otherwise
* DayOfWeek: integer (0-6), 0 monday, day of week
* WeekOfYear: integer, running week within year of this date
* Day integer: day of the date
* Month integer: month of the date
* Year: integer year of the date
* PeriodOfDay: integer denotes half hour period of day (0-47)
* ForecastWindProduction: the forecasted wind production for this period
* SystemLoadEA: the national load forecast for this period
* SMPEA: the price forecast for this period
* ORKTemperature: the actual temperature measured at Cork airport
* ORKWindspeed: the actual windspeed measured at Cork airport
* CO2Intensity: the actual CO2 intensity in (g/kWh) for the electricity produced
* ActualWindProduction: the actual wind energy production for this period
* SystemLoadEP2: the actual national system load for this period
* SMPEP2: the actual price of this time period, the value to be forecasted.


In [ ]:
data.head()

In [ ]:
len(data)

In [ ]:
#create a function to view the data

def Missing_values(data):
    # create list
    variable_name =[]
    total_value = []
    total_missing_value = []
    missing_value_rate =[]
    unique_value_list = []
    total_unique_value =[]
    data_type = []
    
    for col in data.columns:
        variable_name.append(col)
        data_type.append(data[col].dtype)
        total_value.append(data[col].shape[0])
        total_missing_value.append(data[col].isnull().sum())
        missing_value_rate.append(round(data[col].isnull().sum()/data[col].shape[0], 3))
        unique_value_list.append(data[col].unique())
        total_unique_value.append(len(data[col].unique()))
    missing_data = pd.DataFrame({'variable': variable_name,
                                'total value': total_value,
                                'total missing value': total_missing_value,
                                'missing_value_rate': missing_value_rate,
                                'unique_value_list': unique_value_list,
                                'total_unique_value': total_unique_value,
                                'data_type': data_type})
    return missing_data.sort_values('missing_value_rate', ascending=False)

In [ ]:
Missing_values(data)

In [ ]:
# see the unique value
data['Holiday'].unique()

In [ ]:
data['HolidayFlag'].unique()

In [ ]:
data.value_counts(['Holiday']).plot(kind='bar')

In [ ]:
data.value_counts(['HolidayFlag'])

In [ ]:
data.value_counts(['HolidayFlag']).plot(kind='bar', color=['red', 'blue'])

In [ ]:
data.value_counts(['PeriodOfDay']).plot(kind='bar', figsize=(12, 6))

In [ ]:
df_tmp = data.copy()

In [ ]:
# These columns contain strings
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        print(label)

In [ ]:
# This will turn all of the string values into category values
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        df_tmp[label] = content.astype("category").cat.as_ordered()

In [ ]:
# This will turn all of the string values into category values
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        df_tmp[label] = content.astype("category").cat.as_ordered()

In [ ]:
Missing_values(df_tmp)

In [ ]:
# Check columns which *aren't* numeric
for label, content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        print(label)

In [ ]:
# Turn categorical variables into numbers
for label, content in df_tmp.items():
    # Check columns which *aren't* numeric
    if not pd.api.types.is_numeric_dtype(content):
    # We add the +1 because pandas encodes missing categories as -1
        df_tmp[label] = pd.Categorical(content).codes+1

In [ ]:
# Check columns which *aren't* numeric again
for label, content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        print(label)

In [ ]:
Missing_values(df_tmp)

In [ ]:
#Fill the missing value
for label, content in df_tmp.items():
    df_tmp[label] = content.fillna(content.median())

In [ ]:
Missing_values(df_tmp)

In [ ]:
import seaborn as sns
plt.figure(figsize = (12, 5))
sns.distplot(df_tmp['Holiday'])
plt.show()

In [ ]:
plt.figure(figsize = (12, 5))
sns.distplot(df_tmp['ForecastWindProduction'])
plt.show()

In [ ]:
plt.figure(figsize = (12, 5))
sns.distplot(df_tmp['ActualWindProduction'])
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df_tmp['ActualWindProduction'], def_tmp['SMPEP2'])

In [ ]:
a = df_tmp.groupby("Year").sum()["SMPEP2"]
b = df_tmp["Year"].value_counts().index
plt.figure(figsize = (10,6))
sns.barplot(x = b, y = a, color = "blue", ci = None)
plt.title("Prices by years", fontsize = 15)
plt.tick_params(axis = "both", which = "major", labelsize = 12, direction = "out")
plt.xlabel("Years",fontsize = 12)
plt.ylabel("Prices", fontsize = 12)
plt.show()

In [ ]:
a = df_tmp.groupby("Year").mean()["ActualWindProduction"]
b = df_tmp["Year"].value_counts().index
plt.figure(figsize = (10,6))
sns.barplot(x = b, y = a, color = "blue", ci = None)
plt.title("productibility by years", fontsize = 15)
plt.tick_params(axis = "both", which = "major", labelsize = 12, direction = "out")
plt.xlabel("Years",fontsize = 12)
plt.ylabel("ActualWindProduction", fontsize = 12)
plt.show()

In [ ]:
sns.jointplot('CO2Intensity','ActualWindProduction', data=df_tmp,
              kind="kde", fill=True, color='green', height=6);

In [ ]:
g = sns.jointplot(data=df_tmp, x="CO2Intensity", y="ORKTemperature")
g.plot_joint(sns.kdeplot, color="r", zorder=0, levels=6)
g.plot_marginals(sns.rugplot, color="r", height=-.15, clip_on=False)

In [ ]:
sns.jointplot('CO2Intensity','ORKTemperature', data=df_tmp,
              kind="kde", fill=True,
              color='green', height=6);

In [ ]:
#built a function to visulize and compare the data
def plot_fourChart(df, feature1, feature2):
    """
    plot a heatmap to compare the parameters with the actual productivity of energy
    """
    sns.jointplot(df.loc[:,feature1], df.loc[:,feature2], data=df,
              kind="kde", fill=True,
              color='blue', height=6)
    pass

In [ ]:
plot_fourChart(df_tmp, 'ORKWindspeed', 'ActualWindProduction')

the function work good.
now let's try it in a four chart and give it a good visualisation

In [ ]:
# split the data by date
df11 = df_tmp[df_tmp.Year == 2011] #test data
df12 = df_tmp[df_tmp.Year == 2012] #trainig data
df13 = df_tmp[df_tmp.Year == 2013] #valid data
len(df11), len(df12), len(df13)
df11_tmp = df11.copy()
df12_tmp = df12.copy()
df13_tmp = df13.copy()

In [ ]:
len(df11), len(df12), len(df13)


In [ ]:
df13.head()

In [ ]:
FirstPerdDay0 = df11[df11.PeriodOfDay == 0]

In [ ]:
FirstPerdDay0

In [ ]:
plt.figure(figsize = (12, 5))
sns.distplot(FirstPerdDay0['ActualWindProduction'])
plt.show()

In [ ]:
# let's try the midlle periode in the day
FirstPerdDay24 = df11[df11.PeriodOfDay == 24]
FirstPerdDay24

In [ ]:
#The production of the wind in the middle of day
plt.figure(figsize = (12, 5))
sns.distplot(FirstPerdDay24['ActualWindProduction'])
plt.show()

Now let's try to compare the forcasted and the actual data in **2011**
wer're gonna need four plot to visualizethe diffrent.

In [ ]:
#Wind Production
col1= ['ForecastWindProduction', 'ActualWindProduction']
df11_tmp1 = df11[col1]
df11_tmp1

In [ ]:
# the  national system load
col2  = ['SystemLoadEA', 'SystemLoadEP2']
df11_tmp2 = df11[col2]
df11_tmp2

In [ ]:
#The price
col3  = ['SMPEA', 'SMPEP2']
df11_tmp3 = df11[col3]
df11_tmp3

In [ ]:
df11_tmp1[:50].plot.bar();

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 8))

# Index to plot data
ax[0, 0].plot(df11_tmp1); #wind production
ax[0, 1].plot(df11_tmp2); # the  national system load
ax[1, 0].plot(df11_tmp3); # the price

**split the data into train, validation and test sets**

In [ ]:
df_train = df12
df_val = df13
df_test = df11.drop('SMPEP2', axis=1)

In [ ]:
# Split data into X & y
X_train, y_train = df_train.drop("SMPEP2", axis=1), df_train.SMPEP2
X_valid, y_valid = df_val.drop("SMPEP2", axis=1), df_val.SMPEP2

In [ ]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
df_train.shape, df_val.shape, df_test.shape

In [ ]:
df_test.head()

Now all of our data is splited and numeric and there are no missing values, we should be able to build a machine learning model!

## RandomForestRegressor

we'll choose RandomForestRegressor model to predict

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_jobs=-1)

In [ ]:
%%time
model.fit(X_train ,y_train)

In [ ]:
#create evaluation function
from sklearn.metrics import mean_squared_log_error, mean_absolute_error



#create function to evaluate our model
def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {'Training MAE': mean_absolute_error(y_train, train_preds),
             'Valid MAE': mean_absolute_error(y_valid, val_preds),
             'Trainig R^2': model.score(X_train, y_train),
             'valid R^2': model.score(X_valid, y_valid)}
    return scores

In [ ]:
show_scores(model)

## Hyperparameter tuning with RandomizedSearchCV

In [ ]:
%%time
from sklearn.model_selection import RandomizedSearchCV

# Different RandomForestClassifier hyperparameters
rf_grid = {"n_estimators": np.arange(10, 100, 10),
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": np.arange(2, 20, 2),
           "min_samples_leaf": np.arange(1, 20, 2),
           "max_features": [0.5, 1, "sqrt", "auto"],
           "max_samples": [10000]}

rs_model = RandomizedSearchCV(RandomForestRegressor(),
                              param_distributions=rf_grid,
                              n_iter=20,
                              cv=5,
                              verbose=True)

rs_model.fit(X_train, y_train)

In [ ]:
# Find the best parameters from the RandomizedSearch 
rs_model.best_params_

In [ ]:
show_scores(rs_model)

## training model with best params

In [ ]:
%%time
# Most ideal hyperparameters
ideal_model = RandomForestRegressor(n_estimators=70,
                                    min_samples_leaf=13,
                                    min_samples_split=10,
                                    max_features='sqrt',
                                    n_jobs=-1,
                                    max_samples=None)
ideal_model.fit(X_train, y_train)

In [ ]:
show_scores(ideal_model)

## Make prediction
let's make prediction on test dat wich is the data in 2011 and compare it with the true data

In [ ]:
df_test.shape

In [ ]:
test_preds = ideal_model.predict(df_test)
test_preds

In [ ]:
test_preds.shape

In [ ]:
# create a dataFrame for the predicted data
df_preds = pd.DataFrame()
df_preds["predicted Price"] = test_preds
df_preds

In [ ]:
df_preds.shape

In [ ]:
#compare the real price and the predicted one
PriceCol  = ['SMPEP2']
Price = df11[PriceCol]
frames = [Price, df_preds]
result = pd.concat(frames, axis=1)
result.shape, Price.shape

In [ ]:
result[:50].plot.bar();

In [ ]:
result

In [ ]:
(54.32/52.0901)

In [ ]:
rate = result['SMPEP2']/result['predicted Price']

In [ ]:
plt.figure(figsize = (12, 5))
sns.distplot(rate)
plt.show()

* 1 means the perfect predict.
* the most predicted rate here is 90%

In [ ]:
len(Price)

1 mean the perfect predict
the most predicted percentage is 90% 

In [ ]:
# define the important features
ideal_model.feature_importances_

In [ ]:
import seaborn as sns

# Helper function for plotting feature importance
def plot_features(columns, importances, n=100):
    df = (pd.DataFrame({"features": columns,
                        "feature_importance": importances})
          .sort_values("feature_importance", ascending=False)
          .reset_index(drop=True))
    
    sns.barplot(x="feature_importance",
                y="features",
                data=df[:n],
                orient="h")

In [ ]:
plot_features(X_train.columns, ideal_model.feature_importances_)

## RidgeRegressor

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
model2 = Ridge(alpha=0.01)
model2.fit(X_train, y_train)

In [ ]:
show_scores(model2)

In [ ]:
preds2 = model2.predict(df_test)

In [ ]:
preds2, preds2.shape

In [ ]:
# create a dataFrame for the predicted data
df_preds2 = pd.DataFrame()
df_preds2["predicted Price2"] = preds2
df_preds2

In [ ]:
#compare the real price and the predicted one
PriceCol  = ['SMPEP2']
Price = df11[PriceCol]
frames2 = [df_preds2, Price]
result = pd.concat(frames2, axis=1)
result

In [ ]:
rate2 = result['SMPEP2']/result['predicted Price2']

In [ ]:
rate2

In [ ]:
plt.figure(figsize = (12, 5))
sns.distplot(rate2)
plt.show()